In [1]:
# multi-knapsack, integer divisible

import pyomo.environ as pyo

#           item:   value, weight
data = {    1:      (20, 10),
            2:      (30, 20),
            3:      (40, 5),
            4:      (5, 10),
            5:      (100, 10)}
#           bin:    capacity
bins = {    1:      8,
            2:      12,
            3:      14}

prohibited = {(5, 1), (3, 2)}   # (item:bin) that are prohibited.

mdl = pyo.ConcreteModel()

# sets
mdl.invs = pyo.Set(initialize=data.keys())
mdl.bins = pyo.Set(initialize=bins.keys())
mdl.prohibited = pyo.Set(within=mdl.invs*mdl.bins, initialize=prohibited)

# params
mdl.value   = pyo.Param(mdl.invs, initialize= {k:data[k][0] for k in data})
mdl.weight  = pyo.Param(mdl.invs, initialize= {k:data[k][1] for k in data})
mdl.bin_cap = pyo.Param(mdl.bins, initialize= bins)

# vars
mdl.X = pyo.Var(mdl.invs, mdl.bins, domain=pyo.NonNegativeIntegers)     # the amount from invoice i in bin j
mdl.X_used = pyo.Var(mdl.invs, domain=pyo.Binary)

### Objective ###

mdl.OBJ = pyo.Objective(expr=sum(mdl.X[i, b]*mdl.value[i] for 
                        i in mdl.invs for
                        b in mdl.bins), sense=pyo.maximize)

### constraints ###

# don't overstuff bin
def bin_limit(self, b):
    return sum(mdl.X[i, b] for i in mdl.invs) <= mdl.bin_cap[b]
mdl.c1 = pyo.Constraint(mdl.bins, rule=bin_limit)

# all-or-nothing
def use_all(self, i):
    return sum(mdl.X[i, b] for b in mdl.bins) == mdl.X_used[i]*mdl.weight[i]
mdl.c2 = pyo.Constraint(mdl.invs, rule=use_all)

# don't allow prohibited placements
def limit_prohib(self, i, b):
    return mdl.X[i, b] == 0
mdl.c3 = pyo.Constraint(mdl.prohibited, rule=limit_prohib)

# solve it...
solver = pyo.SolverFactory('glpk')
results = solver.solve(mdl)
mdl.X.display()

X : Size=15, Index=X_index
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (1, 1) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (2, 1) :     0 :   6.0 :  None : False : False : NonNegativeIntegers
    (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (2, 3) :     0 :  14.0 :  None : False : False : NonNegativeIntegers
    (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (3, 2) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (3, 3) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (4, 1) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (4, 2) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (4, 3) :     0 :   0.0 :  None : False : False : NonNegativeIntegers
    (5, 1) :     0 :   0.0 :  None : 